In [15]:
!pip install oandapyV20 requests
import requests
import json
import smtplib
from email.mime.text import MIMEText
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.contrib.requests import MarketOrderRequest
from datetime import datetime, timedelta

     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ---------------------------------------- 51.6/51.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69849 sha256=64168b786b2dae62ced2cb823c340a1558a410bae442b44ea44af97525966acb
  Stored in directory: c:\users\lim otto\appdata\local\pip\cache\wheels\d7\4f\5c\9bfd8c96f1a36dccb743e81d8718ccde66454d9cda2427262e
Successfully built oandapyV20


In [16]:
# OANDA API credentials and practice account details
account_id = "your_account_id"
api_key = "your_api_key"
base_url = "https://api-fxpractice.oanda.com"  #replace with live URL for a live account

In [17]:
# Email configuration for automated email notification 
email_address = "limotto0127@gmail.com"
email_password = "xxxxxxxx"

In [18]:
# Function to authenticate with the OANDA API
def authenticate():
    return API(access_token=api_key, environment="practice", request_params={'account_id': account_id})

In [19]:
# Function to get historical candlestick data
def get_historical_data(instrument, granularity, count=10):
    params = {
        "count": count,
        "granularity": granularity
    }
    endpoint = f"/v3/instruments/{instrument}/candles"
    api = authenticate()
    response = api.request(endpoint, params=params)
    return response

In [20]:
# Function to place a market order
def place_market_order(instrument, units):
    data = MarketOrderRequest(instrument=instrument, units=units).data
    endpoint = f"/v3/accounts/{account_id}/orders"
    headers = {'Authorization': f'Bearer {api_key}', 'Content-Type': 'application/json'}
    response = requests.post(f"{base_url}{endpoint}", headers=headers, data=json.dumps(data))
    return response.json()

In [21]:
# Function to send an email
def send_email(subject, body):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_address, email_password)

        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = email_address
        msg['To'] = email_address

        server.sendmail(email_address, email_address, msg.as_string())
        server.quit()
    except Exception as e:
        print(f"Error sending email: {e}")

In [14]:
# Main function for pairing EURUSD based on stratergy H4 on Engulfing Pattern as triger to execute trade
def main():
    instrument = "EUR_USD"
    granularity = "H4"
    historical_data = get_historical_data(instrument, granularity, count=10)

    # Analyze the historical data and implement your trading strategy here
    # For simplicity, let's assume a basic strategy of buying if the last candle closes higher than the previous candle

    if float(historical_data['candles'][-1]['mid']['c']) > float(historical_data['candles'][-2]['mid']['c']):
        print("Executing Buy Order")
        # Calculate take profit and stop loss based on the 2:1 ratio
        current_price = float(historical_data['candles'][-1]['mid']['c'])
        take_profit = current_price + 2 * (current_price - float(historical_data['candles'][-2]['mid']['c']))
        stop_loss = current_price - (current_price - float(historical_data['candles'][-2]['mid']['c']))
           # Adjust the units parameter based on your risk management strategy
        response = place_market_order(instrument, units=1000, take_profit=take_profit, stop_loss=stop_loss)
        print(response)

        # Send an email notification
        subject = "Trade Placed"
        body = "A trade has been placed based on the trading strategy."
        send_email(subject, body)
    else:
        print("No Trade Signal")

if __name__ == "__main__":
    main()


NameError: name 'API' is not defined